Perturbation workflow
---------------------

The following functions implement a workflow for perturbing one gene in a scRNA-seq dataset. The effect of the perturbation is assessed by calculating the impact on the cells' embeddings using Geneformer. 

The workflow works as follows:  
 1. The function *perturbGene* is used to knockdown (set expression to 0) or upregulate (set expression to dataset maximum) of the given gene
 2. The function *embedData* calculates the Geneformer embeddings of a given dataset and calculates a 2D UMAP representation of the Geneformer embedding
 3. The function *calculateDistance* calculates the Euclidean distance of all cells between two embeddings.
 4. The function *knnClustering* identifies the 5 nearest neighbors and assigns the respective cell type label to each cell. 

In [1]:
import pandas as pd
import numpy as np
import anndata as ad
import umap
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
from helical.models.geneformer import Geneformer,GeneformerConfig

INFO:datasets:PyTorch version 2.6.0 available.


In [2]:
def perturbGene(gene, direction, adata, gf):
    """ Perturb gene

    gene: Ensembl gene ID of gene to be perturbed
    direction: either "knowckdown" or "upregulate"; if "knockdown", counts will be set to 0; if "upregulate", counts will be set to the maximum in the matrix
    adata: anndata object
    gf: Geneformer model
    """
    #gene = "ENSG00000188761"
    goiIndex = adata.var.loc[adata.var.ensembl_id == gene].index
    
    if direction == "knockdown":
        adata.X[:, adata.var.loc[adata.var.ensembl_id == "ENSG00000188761"].index] = 0
    elif direction == "upregulate":
        adata.X[:, adata.var.loc[adata.var.ensembl_id == "ENSG00000188761"].index] = adata.X.max()
    
    return adata

def embedData(adata):
    """ Embedding

    Get the Geneformer embedding and the 2D UMAP representation

    adata: anndata object
    """
    # Get Geneformer embeddings
    gfDataset = gf.process_data(adata, gene_names="gene_name")
    gfEmbeddings = gf.get_embeddings(gfDataset)
    
    # 2D UMAP of embedding
    reducer = umap.UMAP(min_dist=0.2, n_components=2, n_epochs=None, n_neighbors=3)
    umap2D = reducer.fit(gfEmbeddings)

    return (gfEmbeddings, umap2D)

def calculateDistance(emb1, emb2):
    """ Euclidean distance between two embeddings

    emb1: first embedding matrix
    emb2: second embedding matrix
    """
    
    return np.linalg.norm(emb1 - emb2, axis=1)

def knnClustering(emb1, emb2, cellTypes):
    """ knn clustering to find differences in neighborhood

    emb1: first embedding matrix
    emb2: second embedding matrix
    cellTypes: cell types
    """
    
    knn = KNeighborsClassifier(n_neighbors=5, metric='cosine')
    knn.fit(emb1, cellTypes)
    perturbCellTypes = knn.predict(emb2)
    confusionMat = confusion_matrix(cellTypes, perturbCellTypes)

    return (perturbCellTypes, confusionMat)